# eQuest Quick Editting Interface

## Introduction

This is toolkit for advanced eQuest users to quickly and simply edit inp files with python. files that eQuest generate in detailed mode.


## Index
- [Get Started](#Get-Started)
    - [1. Install Python/Anaconda](#1.-Install-Python/Anaconda)
    - [2. Explore equest-api](#2.-Explore-equest-api)
- [Quick Demos](#Quick-Demos)
    - [Quickly set up detailed model with space named properly](#qd-1)
    - [Change semi-exterior walls to a certain construction](#qd-2)
- [Classes/Ojbects](#Classes)
    - [`Inp(object)`](#item-1)
    - [`InpList(object)`](#item-2)
    - [`InpItem(object)`](#item-3)
    - [`Title(InpItem)`](#item-4)
    - [`DefaultSetting(InpItem)`](#item-5)
    - [`AirSystem(InpItem)`](#item-6)
    - [`Zone(InpItem)`](#item-7)
    - [`Flr(InpItem)`](#item-8)
    - [`Space(InpItem)`](#item-9)
    - [`Wall(InpItem)`](#item-10)
        - [`InteriorWall`](#item_101)
        - [`ExteriorWall`](#item-102)
        - [`UgWall`](#item-103)
        <br><br>
    - [`Win(InpItem)`](#item-11)
    - [`Door(InpItem)`](#item-12)
    - [`Polygon(InpItem)`](#item-13)
    - [`Pt(object)`](#item-14)

## Get Started
### 1. Install Python/Anaconda
Python is now built-in with the latest Windows 10 system. Try type `python` in command line and see what version of python you have. My system gave me below. <br>
`Python 3.7.0 (v3.7.0:1bf9cc5093, Jun 27 2018, 04:06:47) [MSC v.1914 32 bit (Intel)] on win32
Type "help", "copyright", "credits" or "license" for more information.`
<br><br>
In most cases, instead of runing python files directly using system enviroment, we usually use a "virtual environment" to compile and run the python files. There're lots of benefits of using "virtual environment" and I won't explain too much here. Also there're many different methods to set up a virtual enviroment. To complete our work, I would recommend using [Anaconda](https://www.anaconda.com/). The tool we will be using to call and run python files is called [Jupyter Notebook](https://jupyter.org/). It offers an interactive interface for output, which is perfect for us. 
<br><br>
See below for steps to set up the enviroment and run our first line of python code:
  1. **Installation.** Please download and install free distribution of Anaconda from [here](https://docs.anaconda.com/anaconda/install/windows/). Jupyter Notebook will usually come with the Anaconda installation.
  2. **Open Anaconda Prompt.** Search Windows for "Anaconda Prompt" and left-click on it. You can also use the graphic interface version by searching for "Anaconda Navigator". 
  3. **Set up virtual environment and install dependencies.** In this case, we will just use the base environment (so there's nothing you need to do about it) and install just one package called `bidict`. Make sure you're connect to the internet. Input `pip install bidict` in the propmt window and hit "Enter" then you will be all set. 
  4. **Copy eQuest dependencies.** Copy the fold `\\ENNYCCIFS01\projects\z-MISC\IPA\Resources\eQuest\Python\equest-api` to your local machine.
  5. **Navigate to your work directory.** In the Anaconda Prompt, navigate to the work directory you want to use. The working directly will be the location you will be saving all of your python files but not neccessarily the .inp files. We will also talk about this later. <br> The example to navigate: `cd C:\Users\usxl671686\OneDrive - WSP O365\Research\eQuest Geo API\equest-api`. 
  6. **Open Jupyter Notebook.** Type `Jupyter Notebook` and hit "Enter". The software will open a web page with your default web browser where you can access all the files in your work directory.
  7. **Create your own Notebook.** On the upper right part of the page, click on "New" and select "Python 3" in the dropdown menu. A new tab will pop up and you will have your own Notebook. The file type Jupyter Notebook works on will have extension ".ipynb" which is different from ususal python file extention ".py". Feel free to explore the interface to get familiar with it. Basically each container is called "cell". Each cell can be run seperately and the output will be shown right below the cell. The variables will be saved in the kernel memory once you run the cell and can be called from other cells. To clean up the memory, you will need to restart or shutdown the kernel from the "Kernel" dropdown menu.
  8. **Run "Hello World!".** Type the code below in the first cell and hit Shift+Enter to run the cell.

In [1]:
print("Hello World!")

Hello World!


You can also now open "Get Started with eQuest-api.ipynb", which I used to generate this html file.

 ### 2. Explore equest-api
 I call the package built "equest-api". The abbreviation "api" stands for Application Programming Interface. It kind of doesn't make lots of sense here but we will continue use it until we find another one. In this chapter, I will introduce how eQuest model information a.k.a ".inp" files is stored with equest-api. <br>
 Before that, see the example below to get familiar with some basics of python syntax.

In [23]:
# Comment will start with a "#". Anything after "#" will be ignored.

# This is how functions will be defined.
def func(x):
    print("func(x) is called.")
    return 2*x

# This is what an object looks like
class Point(object):
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        pass
    
    def distance(self, other):
        return ((self.x-other.x)**2 + (self.y-other.y)**2)**0.5
    
# This is some basic syntax
x = 2 # assign 2 to variable x
y = func(2) # call fucntion func(x)
p1 = Point(1, 2) # create an instance of Point
p2 = Point(1, 4) # creatae another instance of Point
dis = p1.distance(p2) # call p1's method Point.distance(other). Note that input "self" is ignored.
print("distance of p1 and p2: ", dis)
print("value of parmater 'x' of p1: ", p1.x) # how to get the value of parameter x of p1

func(x) is called.
distance of p1 and p2:  2.0
value of parmater 'x' of p1:  1


 For more information please refer to [W3 School Python Syntax](https://www.w3schools.com/python/python_syntax.asp). Just go through "Syntax" to "Inheritance" should be enough for us.<br> 
 We will use the eQuest template to illustrate how equest-api works.

First, we need to import the python files using code below. <br>
`import equestapi as eq` <br>
`import equestrun as er` <br>
The file `equestapi` basically contains all the code for reading/editting/saving the inp files. The file `equestrun` will be used to call the DOE-2.2 engine to run the simualtion. See the cell below for how to read the inp file. Reading inp files will also create a backup so that you can go back to the original file if things went wrong.

In [7]:
# import two files
import equestapi as eq
import equestrun as er

# set file location. It can be relative or absolute location
file_loc = r"Project 1\\Project 1.inp" # use \\ as seperator
# read the inp file
inp_file = eq.Inp(file_loc)
# a backup file "Project 1_backup.inp" will be created in the same folder.
inp_list = inp_file.itemList
print(type(inp_file))
print(type(inp_list))

<class 'equestapi.Inp'>
<class 'equestapi.InpList'>


So the `inp_file` is an `Inp` object defined in `equestapi.py`. Two important things about `Inp`: <br>
- `Inp.itemList` - a linked list which contains all building information from the inp file. <br>
- `Inp.save(location = None, inplace = False)` - save the inp file. By default, `inplace` is `False`, which means a new file will be created instead overwriting the existing file. If `location` is not assigned, the program will just save the file with suffix "vx" where x will be the version of the new file.

The object [`InpList`](#item-2) contains a linked list of all building information in the inp file. Every individual block/title block is a node in the linked list. Each node is an [`InpItem`](#item-3) or its child object. Most of them are seperated by ".." in the inp file and some titles will start with "$". This data structure ensure the order of each block in the inp file when we make changes to them. Such block and title block are named "item".<br>
See below for the details of how building information is stored in python object.

In [17]:
first_item = inp_list.head
sec_item = first_item.next
third_item = sec_item.next
forth_item = third_item.next
fifth_item = forth_item.next
sixth_item = fifth_item.next
print("1st node item type: ",type(first_item), "\n1st node item content: \n\t", str(first_item.inp_text).replace("\n", "\n\t "))
print("2nd node item type: ",type(sec_item), "\n2nd node item content: \n\t", str(sec_item).replace("\n", "\n\t "))
print("3rd node item type: ",type(third_item), "\n3rd node item content: \n\t", str(third_item).replace("\n", "\n\t "))
print("4th node item type: ",type(forth_item), "\n4th node item content: \n\t", str(forth_item).replace("\n", "\n\t "))
print("5th node item type: ",type(fifth_item), "\n5th node item content: \n\t", str(fifth_item).replace("\n", "\n\t "))
print("6th node item type: ",type(sixth_item), "\n6th node item content: \n\t", str(sixth_item.inp_text).replace("\n", "\n\t "))

1st node item type:  <class 'equestapi.InpItem'> 
1st node item content: 
	 INPUT ..
	 
2nd node item type:  <class 'equestapi.Title'> 
2nd node item content: 
	 $ ---------------------------------------------------------
	 $              Abort, Diagnostics
	 $ ---------------------------------------------------------
	 
3rd node item type:  <class 'equestapi.Title'> 
3rd node item content: 
	 $ ---------------------------------------------------------
	 $              Global Parameters
	 $ ---------------------------------------------------------
	 
4th node item type:  <class 'equestapi.Title'> 
4th node item content: 
	 $ ---------------------------------------------------------
	 $              Title, Run Periods, Design Days, Holidays
	 $ ---------------------------------------------------------
	 
5th node item type:  <class 'equestapi.InpItem'> 
5th node item content: 
	 TITLE           
	    LINE-1           = *Project 1*
	    ..
	 
6th node item type:  <class 'equestapi.InpIte

`InpList` -  also have some other properties and methods, detailed [below](#item-2). <br>
`InpItem` - is the parent object for all kinds of blocks and title blocks in the inp file. It also works as a node in the `InpList`(it basically has an parameter `prev` which point to the previous node and `next` which point to next node). All the child objects like [`Title(InpItem)`](#item-4) and [`AirSystem(InpItem)`](#item-6) will inherite the properties and methods of `InpItem` and the child object will have some special paramters and methods. See below for some sample code about how to access items.

In [25]:
# Get all item types/key words, which is the string behind the "=" at the first line of each block.
# Title blocks will be counted as 'SS'.
print(inp_list.get_all_types())

{'BUILD-PARAMETERS', 'GLASS-TYPE', 'CIRCULATION-LOOP', 'BASELINE', 'POLYGON', 'MASTER-METERS', 'PUMP', 'TITLE', 'SPACE', 'RUN-PERIOD-PD', 'CONSTRUCTION', 'SCHEDULE-PD', 'DW-HEATER', 'HOURLY-REPORT', 'SYSTEM', 'ELEC-METER', 'WEEK-SCHEDULE-PD', 'DAY-SCHEDULE-PD', 'LAYERS', 'BOILER', 'SS', 'FLOOR', 'ZONE', 'WINDOW', 'INTERIOR-WALL', 'SITE-PARAMETERS', 'EXTERIOR-WALL', 'HOLIDAYS', 'HEAT-REJECTION', 'MATERIAL', 'UNDERGROUND-WALL'}


There're two ways to access all items by its type. For the space below:<br>
`"EL1 South Perim Spc (G.S1)" = SPACE`<br>`...`<br>
We can either acess by the object type `eq.Space` or the key word `SPACE` (see below). Please note that if the item type is not implement yet, only by key word works.

In [28]:
# get all spaces by object type
all_spaces = inp_list.get_item_by_class(eq.Space)
# print all space names
for space in all_spaces:
    print(space.name)

EL1 South Perim Spc (G.S1)
EL1 East Perim Spc (G.E2)
EL1 North Perim Spc (G.N3)
EL1 West Perim Spc (G.W4)
EL1 Core Spc (G.C5)


In [30]:
# get spaces by key word
all_spaces = inp_list.get_item_by_type("SPACE")
# print all space names
for space in all_spaces:
    print(space.name)

EL1 South Perim Spc (G.S1)
EL1 East Perim Spc (G.E2)
EL1 North Perim Spc (G.N3)
EL1 West Perim Spc (G.W4)
EL1 Core Spc (G.C5)


Take `eq.Space` as an example to illustrate a typical `InpItem`'s paramters.

In [41]:
# Typical InpItem (takes Space as an example)
space1 = all_spaces[0] # the first space in the file

# all InpItem's common paramters
print("Common Parameters:")
print("Inp Text: \n", space1.inp_text.replace("\n", "\t"))
print("Inp Item Name: ", space1.name)
print("Inp Item Type: ", space1.iType)
print("Inp List: ", space1.inp_list) # the inp list that the InpItem belongs to
print("Previous Item: ", space1.prev) # next item
print("Next Item: ", space1.next) # next item
# Space's parameters
print("\nSpace Special Paramters:")
print("Thermal zone space belongs to: ", space.zone)
print("Space Activity Type: ", space.activity_type)
print("Conditioned Type: ", space.conditioned)
print("Walls: ", space.walls) # including roofs, walls, and flrs
print("Roof: ", space.roofs)

Common Parameters:
Inp Text: 
 "EL1 South Perim Spc (G.S1)" = SPACE           	   SHAPE            = POLYGON	   ZONE-TYPE        = CONDITIONED	   PEOPLE-SCHEDULE  = "EL1 Bldg Occup Sch"	   LIGHTING-SCHEDUL = ( "EL1 Bldg InsLt Sch" )	   EQUIP-SCHEDULE   = ( "EL1 Bldg Misc Sch" )	   INF-SCHEDULE     = "ZG0-S1 (PVVT) P-Inf Sch"	   INF-METHOD       = AIR-CHANGE	   INF-FLOW/AREA    = 0.0566793	   PEOPLE-HG-LAT    = 130.53	   PEOPLE-HG-SENS   = 245.478	   EQUIP-LATENT     = ( 0 )	   EQUIP-SENSIBLE   = ( 1 )	   LIGHTING-W/AREA  = ( 2.27884 )	   EQUIPMENT-W/AREA = ( 0.407 )	   AREA/PERSON      = 72.5702	   POLYGON          = "EL1 Space Polygon 1"	   LOCATION         = FLOOR-V1	   C-SUB-SRC-BTUH   = ( 0, 0, 0 )	   C-SUB-SRC-KW     = ( 0, 0, 0 )	   C-ACTIVITY-DESC  = *Religious Worship (40%)*	   ..	
Inp Item Name:  EL1 South Perim Spc (G.S1)
Inp Item Type:  SPACE
Inp List:  <equestapi.InpList object at 0x0000012EF58B56D8>
Previous Item:  FLOOR: EL1 Ground Flr
Next Item:  EXTERIOR-WALL: EL1 South

## Quick Demos
<a id="qd-1"></a>
### 1. Quickly set up detailed model based on space name
This code below was used for 223 23rd st project. I put it here just for a reference about the general work flow to setup a model quickly. Check "zonetypeslookup.csv" and "inputsample.py" for the details of some how input information is formatted. This is also the first project that used the interface to setup the energy model and obviously there're much better ways to do it within the interface built.

In [ ]:
# Setting Inputs
import sys
file_loc = r"C:\Users\usxl671686\Desktop\N19 223 23rd St\Energy Model\Test\223 23rd St DOAS.inp"
lookup_loc = r"zonetypeslookup.csv"
input_loc = r"C:\Users\usxl671686\Desktop\N19 223 23rd St\Energy Model\Resources"
baseline_loc = r"C:\Users\usxl671686\Desktop\N19 223 23rd St\Energy Model\Test\223 23rd St Baseline.inp"
sys.path.insert(1, input_loc)
from inputsample import data
from inputsample import zone_data
# End of Inputs

# read .inp file
inp = eq.Inp(file_loc)
itemList = inp.itemList
spaces = itemList.get_item_by_type("SPACE")
print("223 23rd St Model has: {} zones".format(len(spaces)))
# change space type
lookup_table = pd.read_csv(lookup_loc)
lookup = eq.Space.parse_lookup_table(lookup_table)
for space in spaces:
    space.change_act_type_by_name(lookup)
print("Space types has been changed to eQuest code Type.")
# Pass Test: True

# set occ, lighting, equipment to default, except equipment load in apt.
occ = r"AREA/PERSON"
light = r"LIGHTING-W/AREA"
equip = r"EQUIPMENT-W/AREA"
for space in spaces:
    space.reset_default(occ)
    space.reset_default(light)
    if space.activity_type != "*APT*":
        space.reset_default(equip)

# add user-defined default to occ, light, equip
titles = itemList.get_item_by_type("TITLE")
for t in titles:
    if t.name == "Floors / Spaces / Walls / Windows / Doors":
        itemList.add_default_settings(t, "SPACE", data)
        
# system inputs changes
# remove fan power 
systems = itemList.get_item_by_type("SYSTEM")

for sys in systems:
    sys.reset_default("SUPPLY-STATIC")
    sys.reset_default("SUPPLY-EFF")
    sys.reset_default("RETURN-EFF")
    sys.reset_default("SIZING-RATIO")
for sys in systems:
    if sys.sysType == "PVVT" or sys.sysType == "PTAC":
        sys.set_fan_power(0.0003)
        sys.heat_sizing_ratio = 1.25
        sys.cool_sizing_ratio = 1.15
        sys.night_cycle = "CYCLE-ON-ANY"
    # add DOAS settings for sys2, sys7, sys8, sys9, sys12
    sys_name = sys.name
    if "Sys2" in sys_name or "S7" in sys_name or "S8" in sys_name \
    or "S9" in sys_name or "Sys12" in sys_name:
        sys.oa_from_sys = '"S1 Sys (PVVT)"'
    
# zone inputs changes
zones = itemList.get_item_by_type("ZONE")
for zone in zones:
    zone.reset_default("OA-FLOW/PER")

# add default settings
titles = itemList.get_item_by_type("TITLE")
for t in titles:
    if t.name == "HVAC Systems / Zones":
        itemList.add_default_settings(t, "ZONE", zone_data)

#Remove Zone Heat Source for Apartment and Corridor WSHP
for sys in systems:
    if "Sys2" in sys.name or "Sys12" in sys.name:
        sys._set_value("ZONE-HEAT-SOURCE", "NONE")
        
# Correct Equipment Load for Dwelling Unit
# In-Unit misc: 0.5 w/sf
# Refreg: 0.262 w/sf
# Stove: 0.374 w/sf
# Washer and Dishwasher: 0.035+0.102 = 0.137 w/sf
spaces = itemList.get_item_by_type("SPACE")
for s in spaces:
    if s.activity_type == "*APT*":
        s._set_value("EQUIPMENT-W/AREA", "( &D, 0, 0, 0, 0 )")
# Do the same for baseline model        
inp_base = eq.Inp(baseline_loc)
itemList_base = inp_base.itemList
spaces = itemList_base.get_item_by_type("SPACE")
for s in spaces:
    if s.activity_type == "*APT*":
        s._set_value("EQUIPMENT-W/AREA", "( &D, 0, 0, 0, 0 )")   

# Set Baseline WWR to 40%
wins = items_base.get_item_by_type("WINDOW")
for win in wins:
    h = win._get_value("HEIGHT")
    win._set_value("HEIGHT", """{{{height}*#pa("WRatio")}}""".format(height = h))
    
# Change OA Schedule
# proposed
systems = itemList.get_item_by_type("SYSTEM")
for s in systems:
    if s._get_value("TYPE") == "PVVT":
        s.reset_default("MIN-AIR-SCH")
# baseline
systems = itemList_base.get_item_by_type("SYSTEM")
for s in systems:
    if s._get_value("TYPE") == "PTAC":
        s.reset_default("MIN-AIR-SCH")
        
# Change proposed heatitng EIR
systems = itemList.get_item_by_type("SYSTEM")
for s in systems:
    if s._get_value("TYPE") == "PVVT":
        s._set_value("HEATING-EIR", '{#pa("HeatingEIR")}')

# save both files
inp_base.save(inplace = True)
inp.save(inplace = True)

<a id="qd-2"></a>
### 2. Change semi-exterior walls to a certain construction
**Warning Info: ** "unconditioned {} {} was connected to conditioned {} {}. Revise before proceeding." <br>
**Solution: ** make sure zone/space is consistent.

In [2]:
# Demo 2

# import package
import equestapi as eq
import equestrun as er

# loading inp file
inp_file = r"Project 1\\Project 1.inp"
inp = eq.Inp(inp_file)
item_list = inp.itemList

# set up semi-xterior wall construction name
cons_name = "EL1 EWall Construction"

# run methods
item_list.enhance_semi_exterior_wall(cons_name)

unconditioned space:  [SPACE: EL1 North Perim Spc (G.N3)]
unconditioned_zones:  [ZONE: EL1 North Perim Zn (G.N3)]
INTERIOR-WALL: EL1 NW Wall (G.E2.I4) was modified to semi-exterior construction.
INTERIOR-WALL: EL1 SW Wall (G.N3.I6) was modified to semi-exterior construction.
INTERIOR-WALL: EL1 South Wall (G.N3.I7) was modified to semi-exterior construction.


True

## Classes [To Be Updated]
<a id="item-1"></a>
1. `Inp(object)`
    - **Parameters** 
        - `file_loc`<br>File location of the .inp file.
        - `version` - TBD
        - `itemList` - `InpList`<br> Contains all items in the .inp file in a linked list.
    - ** Methods**
        - `create_rectangle_shade`
        - `create_global_parameter(name=string, value=float/int/double)`<br>create a global parameter in the .inp file.
        - `save(name=str, inplace=False)`<br>save inp. file.
<br>
<br>
<a id="item-2"></a>
2. `InpList(object)`
    - **Parameters**
    - ** Methods**
        - `get_all_types(self)`<br>return all item types of the inp. file.
        - `get_item_by_type(self, item_type=str)`<br>get all items in the inp. file by the type.
        - `insert_after(self, prev_node=InpItem, new_item=InpItem)`<br>insert new item to the inp. file.
        - `printList(self)`<br>
        - `add_default_settings(self, title=Title, item_type=str, data=dict())`<br> add defualt setting to the inp file. `title` is the type object to add. <br><br>
<a id="item-3"></a>
3. `InpItem(object)`
    - **Params**
        - `name` <br> name of the inp item.
        - `iType` <br> item type.
    - ** Methods**
        - `_get_value(self, key=str)` <br> \[private\] get value by attribute name.
        - `_set_value(self, key=str, value=str)` <br> \[private\] set value by attribute name.
        - `_get_type_and_name(self)` <br> \[private\] return item type and name.
        - `change_name(self, new_name=str)` <br> change name of the item.
        - `reset_default(self, attr_name=str)` <br> reset attribute value to default.
       <br><br>
       
<a id="item-4"></a>
4. `Title(InpItem)` 
<br> 
<br><br>
<a id="item-5"></a>
5. `DefaultSetting(InpItem)`
<a id="item-6"></a>
6. `AirSystem(InpItem)`
<a id="item-7"></a>
7. `Zone(InpItem)`
<a id="item-8"></a>
8. `Flr(InpItem)`
<a id="item-9"></a>
9. `Space(InpItem)`
<a id="item-10"></a>
10. `Wall(InpItem)`
<a id="item-11"></a>
11. `Win(InpItem)`
<a id="item-12"></a>
12. `Door(InpItem)`

** Appendix** <br>
**All eQuest Item Types**<br>
Abort, <br>
Diagnostics, <br>
Global Parameters,<br>
Title,<br>
Run Periods,<br>
Design Days,<br>
Holidays,<br>
Compliance Data, <br>
Site, <br>
Building Data,<br>
Materials -> Layers -> Constructions<br>
Glass Type Codes, <br>
Glass Types, <br>
Window Layers,<br>
Lamps -> LUminaries -> Lighting Systems, <br>
Day Schedules -> Week Schedules -> Annual Schedules, <br>
Polygons,<br>
Wall Parameters,<br>
Fixed and Building Shades,<br>
Misc Cost Related Objects,<br>
Performance Curves,<br>
Floors <- Spaces <- Walls <- Windows <- Doors<br>
Electric and Fuel Meters<br>
Electric Meters<br>
Master Meters<br>
HVAC Circulation Loops <- Plant Equipment<br>
Pumps<br>
Heat Exchangers<br>
Circulation Loops<br>
Chillers<br>
Boilers <br>
Domestic Water Heaters<br>
Heat Rejection<br>
Tower Free Cooling<br>
Phtovoltaic Modules<br>
Electric Generators<br>
Thermal Storage<br>
Ground Loop Heat Exchangers<br>
Compliance DHW (Residential dwelling unit)<br>
Steam and Chilled Water Meters<br>
Steam Meters<br>
Chilled Water Meters<br>
HVAC Systems <- Zones<br>
Metering and Misc HVAC<br>
Equipment Controls<br>
Load Management<br>
Utility Rates<br>
Ratchets<br>
Block Charges<br>
Utility Rates<br>
Output Reporting<br>
Loads Non-Hourly Reporting<br>
Systems Non-Hourly Reporting<br>
Plant Non-Hourly Reporting<br>
Economics Non-Hourly Reporting<br>
Hourly Reporting<br>
The End<br>
